# **Océrisation d'un dossier de fichiers JPG grâce à l'API de Google Vision**

Pour exécuter ce notebook, il est nécessaire :
- d'avoir un compte Google Cloud dans lequel est stocké les fichiers JPG dans un bucket public Google Storage. Pour rendre public un bucket, suivre les [instructions suivantes](https://cloud.google.com/storage/docs/access-control/making-data-public?hl=fr).
- d'avoir accédé au service Google Vision de Google Cloud et d'avoir généré une clé d'API Google Vision
- d'avoir les mêmes fichiers JPG stockés dans un dossier sur son Google Drive

Toutes les langues reconnues par Google Vision sont disponibles [ici](https://cloud.google.com/vision/docs/languages?hl=fr).

*Résultat* : récupération du texte océrisé des fichiers jpg et de la langue détectée avec son taux de fiabilité dans un fichier Excel.

In [ ]:
#@markdown # Connexion du notebook à un compte Google Drive

#@markdown - Lancer la cellule
#@markdown - Cliquer sur « Exécuter malgré tout » lors de l’apparition du message d’avertissement indiquant que le notebook n’a pas été créé par Google
#@markdown - Cliquer sur « Se connecter à Google Drive » lors de l’apparition du second message d’avertissement pour donner l’autorisation au notebook d’accéder à vos fichiers Google Drive
#@markdown - Choisir son compte Gmail puis cliquer sur « Autoriser »

from google.colab import drive
import os
import pandas as pd
from os import listdir
from os.path import isfile, join

drive.mount('/content/drive/')
%cd /content/drive/My Drive/

In [ ]:
#@markdown # Installation des packages nécessaires

#@markdown Exécuter simplement la cellule.

!pip install --upgrade pip
!pip install --upgrade google-api-python-client

In [ ]:
#@markdown # Récupération des noms des fichiers jpg à partir d'un dossier Google Drive

#@markdown Indiquer le chemin vers le dossier d'images sur le Google Drive :
chemin_vers_dossier_dimages = '/content/drive/My Drive/Images/'#@param {type:"string"}

#@markdown En résultat s'affiche le nombre de fichiers jpg contenus par le dossier.

liste_fichiers = [f for f in listdir(chemin_vers_dossier_dimages) if isfile(join(chemin_vers_dossier_dimages, f))]
print(len(liste_fichiers))

# Création d'un dataframe des images traitées

df = pd.DataFrame(liste_fichiers, columns = ['Image traitée'])
df = df.sort_values(by=['Image traitée'])

In [ ]:
#@markdown # Lancement de l'océrisation

#@markdown Indiquer la clé d'API Google Vision :
APIKEY = ''#@param {type:"string"}

#@markdown Indiquer le nom du bucket Google Storage dans lequel se trouve les fichiers jpg (attention les fichiers doivent se trouver directement dans le bucket et non dans un sous-dossier) :
#
nom_bucket = ''#@param {type:"string"}

#@markdown Indiquer le chemin du fichier Excel de sortie :
chemin_fichier_resultats = "/content/drive/MyDrive/resultat_ocr.xlsx"#@param {type:"string"}

from googleapiclient.discovery import build
import base64

# Pour chaque fichier dans le lot de fichier créé auparavant, création de l'URI du fichier dans le bucket "affiches" de Google Storage
for nom in liste_fichiers:
  IMAGE ="gs://" + nom_bucket + "/" + nom

# Lien avec le service et envoie de la requête à l'API Google Vision
  vservice = build('vision', 'v1', developerKey=APIKEY)
  request = vservice.images().annotate(body={
        'requests': [{
                'image': {
                    'source': {
                        'gcs_image_uri': IMAGE
                    }
                },
                'features': [{
                    'type': 'TEXT_DETECTION',
                    'maxResults': 3,
                }]
            }],
        })

  # Récupération de la réponse de la requête
  reponse = request.execute(num_retries=3)

  # Récupération du texte intégral de l'affiche
  try:
    texte = reponse['responses'][0]['textAnnotations'][0]['description']
  except KeyError:
    texte = "Le fichier jpg ne contient pas de contenu textuel."

  # Récupération du code de la ou les langues de l'affiche avec son ou ses taux de fiabilité
  langues = []
  try:
    for l in reponse['responses'][0]['fullTextAnnotation']['pages'][0]['property']['detectedLanguages']:
      langue = l['languageCode'], ":", str(l['confidence'])
      langue = ' '.join(langue)
      langues.append(langue)
  except KeyError:
    langue = "La langue du fichier jpg n'a pas pu être détectée."
    langues.append(langue)
  

  # Récupération des informations récoltées suivant l'affiche correspondante dans un nouveau fichier Excel
  for nom_fichier in df['Image traitée']:
    if IMAGE.__contains__(nom_fichier):
      matched_row = df.index[df['Image traitée'] == nom_fichier]
      df.at[matched_row, 'Texte'] = str(texte)
      df.at[matched_row, 'Langue(s) et taux de fiabilité'] = str(langues)
      print(nom_fichier)

df.to_excel(chemin_fichier_resultats)